In [ ]:
import requests, json
from pprint import pprint

# base url for all FPL API endpoints
url1 = 'https://fantasy.premierleague.com/api/bootstrap-static/'
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r1 = requests.get(url1).json()

# show the top level fields
pprint(r1, indent=1, depth=1)

In [ ]:
# get player data from 'elements' field
# players = r1['elements']
# show data for first player
# pprint(players[124])

pprint(r1['elements'], indent=1, depth=2)

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
# create players dataframe
players = pd.json_normalize(r1['elements'])

# show some informatidon about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

In [ ]:
# create teams dataframe
teams = pd.json_normalize(r1['teams'])
teams.head()

In [ ]:
# get position information from 'element_types' field
positions = pd.json_normalize(r1['element_types'])
positions.head()

In [ ]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

In [ ]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

In [ ]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

In [ ]:
# show data for first gameweek
pprint(r['history'][1])

In [ ]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
get_gameweek_history(4)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head()

In [ ]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

In [ ]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

In [14]:
from tqdm.auto import tqdm
tqdm.pandas()

In [15]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/714 [00:00<?, ?it/s]

In [ ]:
# get top scoring players
# test_df =
points.groupby(
                    ['element', 'web_name']
                ).agg(
                    {'total_points':'sum',
                     'goals_scored':'sum', 
                     'assists':'sum',
                     'selected':'sum'}
                ).reset_index(
                ).sort_values(
                    'total_points', ascending=False
                ).head()

In [17]:
import os  
os.makedirs('desktop/FPL', exist_ok=True)  
points.to_csv('desktop/FPL/points.csv')  
players.to_csv('desktop/FPL/players.csv')  

In [18]:
import psycopg2
import io
import pandas as pd

In [19]:
# Create a connection to your PostgreSQL database
conn = psycopg2.connect(host="localhost", database="databasename", user="username", password="password")

# Create a cursor object
cur = conn.cursor()


In [47]:
cur.execute("TRUNCATE TABLE squad, points, player;")

In [48]:
# Create a DataFrame with your data
df = pd.DataFrame(players)

# Create a buffer to write the DataFrame to
buffer = io.StringIO()

# Write the DataFrame to the buffer as a CSV file
df.to_csv(buffer, sep=',', index=False, header=False)

# Reset the buffer's position to the start
buffer.seek(0)

# Use the copy_from method to load the CSV file into a PostgreSQL table
cur.copy_from(buffer, 'player', sep=',')

# Commit the changes to the database
conn.commit()

# Close the cursor and connection objects
cur.close()
conn.close()